<a href="https://colab.research.google.com/github/AnIsAsPe/Estadistica_y_Probabilidad_para-CD-/blob/main/Semana6/Metropolis_Hastings_para_decifrar_textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2> Algoritmo Metropolis Hastings para decifrar textos <h2>

El presente código es una adaptación del presentado en [este repositorio](https://github.com/svivek/mcmc-decoding-example)

El ejemplo está inspirado en la historia descrita en [The Markov Chain Monte Carlo Revolution](https://www.ams.org/journals/bull/2009-46-02/S0273-0979-08-01238-X/S0273-0979-08-01238-X.pdf). 

El objetivo es decifrar los mensajes intercambiados por prisioneros, con la conjetura de que cada simbolo representa una letra o simbolo del lenguaje, de manera que el objetivo es encontrar ese diccionario o **clave** para codificar y decodificar los mensajes.





<center><img src='https://pbs.twimg.com/media/DnJnUCpWwAAmPnR.jpg'>



## Cifrado por sustitución

Es un método de cifrado para codificarr y decodificar mensajes usando una clave que relaciona una letra o simbolo por otro.

Como ejemplo, se puede suponer que la clave de un cifrado está contenida en la siguiente tabla:

|original | codificado |
|---------|------------|
|a        | z          |
|n        | 4          |
|l        | 5          |
|i        | a          |
|s        | 8          |

De manera que la palabra  `analisis` puede ser codificada como `z4z5a8a8`. 

In [1]:
def code(text, key): 
    return ''.join(list(map(lambda char: key[char], text)))

In [2]:
clave = {'a': 'z', 'n': '4', 'l': '5', 'i': 'a', 's':'8'}
code('analisis', clave)

'z4z5a8a8'

Y podemos decodificar la palabra usando la clave inversa:

In [3]:
inv_clave = {v : k for k, v in clave.items()}
code('z4z5a8a8', inv_clave)

'analisis'

#¿Cómo encontrar una clave adecuada?

Si consideramos solamente el espacio en blanco, las letras en minúscula, y los números estamos hablando de 38 simbolos, y por tanto el espacio muestral en el que vive la clave es enorme.

In [4]:
alphabet = "abcdefghijklmnñopqrstuvwxyz0123456789 "

print('Total de simbolos considerados:', len(alphabet))

from math import factorial
print('Tamaño del espacio muestral en el que vive la clave que buscamos', factorial(38))

Total de simbolos considerados: 38
Tamaño del espacio muestral en el que vive la clave que buscamos 523022617466601111760007224100074291200000000


Supondremos que el lenguaje de los mensajes cifrados es el español, por tanto utilizaremos como guía las probabilidades de transición de los carácteres en español.

# Calificación de las claves candidatas

Para cada secuencia de caracteres $c_1c_2c_3\cdots c_n$, estimaresmos su probabilidad de la siguiente manera

$$P(c_1c_2c_3\cdots c_n) = P(c_1) P(c_2 \mid c_1) P(c_3 \mid c_2) \cdots P(c_n \mid c_{n-1})$$

Así, la probabilidad de la palabra `analisis` se calcularía como 

 $P(a)~P(n \mid a)P(a \mid n)P(l \mid a)P(i \mid l)P(s \mid i)P(i \mid s)P(s \mid i)$.

Para hacer lo anterior necesitamos una matriz de transición de una caracter a otro, y en este sentido, para facilitar el análisis,  ignoraremos el primer caracter porque no hará practicamente diferencia en una secuencia larga.

## ¿Cómo obtener la matriz de transición de los caracteres? 


In [5]:
# ¿Cuántas probabilidades de transición debe tener nuestra matriz de trancisión?
len(alphabet)**2

1444

Para aprender estas probabilidades usaremos un texto, lo más largo posible en español

In [6]:
import re

def limpiar_texto(texto):

    #cambiar texto a minusculas
    texto = texto.lower()
    
    # quitar tildes
    a,b = 'áéíóúü','aeiouu'
    trans = str.maketrans( a,b)
    texto = texto.translate(trans)

    # solo letras y numeros
    texto = re.sub('[^ña-z0-9 ]+' ,' ', texto).strip()  

    # remplazar multiples espacios contiguos por un solo espacio
    texto = re.sub('\s+',' ', texto)                    
    return texto

def find_between( s, first, last ):
    '''fuente: https://stackoverflow.com/questions/3368969/find-string-between-two-substrings'''
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""
def leer_texto(file, inicio=None, fin=None):
    with open(file, 'r') as f:
        if inicio is not None and fin is not None:
            libro = find_between(f.read(), inicio, fin)
        else:
            libro = f.read()
    return limpiar_texto(libro)

Se puede calcular las probabilidades de transisión utilizando cualquier texto. Si este es suficientemente grande, podremos obtener buenas estimaciones. Usaremos Don Quijote de la Mancha, que tiene más de 2 millones de caracteres.

In [7]:
from collections import Counter
import math

def bigram_log_probabilities(book):
    
    character_counts = Counter(book)
    bigram_counts = Counter(zip(book, book[1:]))
    bigrams = bigram_counts.keys()
    
    return dict(map(lambda k: [k, math.log(bigram_counts[k]) - math.log(character_counts[k[0]])], bigrams))

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
inicio = "*** START OF THE PROJECT GUTENBERG EBOOK DON QUIJOTE ***"
fin = "*** END OF THE PROJECT GUTENBERG EBOOK DON QUIJOTE ***"

file = '/content/drive/MyDrive/Datos/DonQuijote.txt'
DonQ = leer_texto(file, inicio, fin )

In [10]:
bigram_table = bigram_log_probabilities(DonQ)
bigram_table

{('e', 'l'): -2.495635370118805,
 ('l', ' '): -1.633573260510028,
 (' ', 'i'): -4.937384512589171,
 ('i', 'n'): -2.3504318062150116,
 ('n', 'g'): -4.031173508631302,
 ('g', 'e'): -2.980174879019078,
 ('e', 'n'): -1.8827851415801167,
 ('n', 'i'): -3.196628842521836,
 ('i', 'o'): -2.349621480303913,
 ('o', 's'): -1.8348327937786753,
 ('s', 'o'): -2.8913442783423715,
 ('o', ' '): -0.8242763995237041,
 (' ', 'h'): -3.420071451532097,
 ('h', 'i'): -2.3957986266629003,
 ('i', 'd'): -2.801143855311272,
 ('d', 'a'): -2.023135023232147,
 ('a', 'l'): -2.6003410610565005,
 ('l', 'g'): -4.105093437001927,
 ('g', 'o'): -1.4025662217672572,
 (' ', 'd'): -2.166409555430894,
 ('d', 'o'): -1.3743360383401892,
 ('o', 'n'): -2.0723476075124374,
 ('n', ' '): -1.137340896369082,
 (' ', 'q'): -2.633056752874694,
 ('q', 'u'): 0.0,
 ('u', 'i'): -2.480561604574996,
 ('i', 'j'): -2.933481589751146,
 ('j', 'o'): -0.5809899206883387,
 ('o', 't'): -3.485054422488467,
 ('t', 'e'): -1.3776567950215597,
 ('e', ' '): 

Y mediante la siguiente función calcularemos que tan "buena" es la secuencia utilizando para ello las probabilidades de transición estimadas.

In [11]:
LOG_EPSILON = -30  # probabilidad muy pequeña vista como logaritmo, que se ustilizara cuando no exista probabilidad de trancisión de una letra a otra

def goodness(text, bigram_table):
    bigrams = zip(text, text[1:])
    sum = 0
    for b in bigrams:
        sum += bigram_table.get(b) or LOG_EPSILON
    return sum

In [12]:
print("Log probability of 'opamc' = " +str(goodness("opamc", bigram_table)))

Log probability of 'opamc' = -40.784560457172915


In [13]:
print("Log probabilidad de 'campo' = " +str(goodness("campo", bigram_table)))

Log probabilidad de 'campo' = -9.40201613643682


# Implementación

## Creación del mensaje encriptado

In [49]:
plain_text = limpiar_texto(
    """Muchos años después, frente al pelotón de fusilamiento, el coronel Aureliano Buendía había de 
recordar aquella tarde remota en que su padre lo llevó a conocer el hielo. Macondo era entonces 
una aldea de veinte casas de barro y cañabrava construidas a la orilla de un río de aguas diáfanas 
que se precipitaban por un lecho de piedras pulidas, blancas y enormes como huevos 
prehistóricos. El mundo era tan reciente, que muchas cosas carecían de nombre, y para 
mencionarlas había que señalarías con el dedo. Todos los años, por el mes de marzo, una familia 
de gitanos desarrapados plantaba su carpa cerca de la aldea, y con un grande alboroto de pitos y 
timbales daban a conocer los nuevos inventos. Primero llevaron el imán. Un gitano corpulento, de 
barba montaraz y manos de gorrión, que se presentó con el nombre de Melquíades, hizo una 
truculenta demostración pública de lo que él mismo llamaba la octava maravilla de los sabios 
alquimistas de Macedonia. Fue de casa en casa arrastrando dos lingotes metálicos, y todo el 
mundo se espantó al ver que los calderos, las pailas, las tenazas y los anafes se caían de su sitio, 
y las maderas crujían por la desesperación de los clavos y los tornillos tratando de desenclavarse, 
y aun los objetos perdidos desde hacía mucho tiempo aparecían por donde más se les había 
buscado, y se arrastraban en desbandada turbulenta detrás de los fierros mágicos de Melquíades. 
"""
)

In [50]:
import random

def make_random_key():
    l = list(alphabet)
    random.shuffle(l)
    return dict(zip(alphabet, l))

In [51]:
key = make_random_key()
cipher_text = code(plain_text, key)
cipher_text

'qrfoi5l1gi5lxe58re5l4temdel1ñl8eñidimlxel4r52ñ1q2emdileñlfitimeñl1rteñ21milvremx21lo1v21lxeltefitx1tl17reññ1ld1txelteqid1leml7rel5rl81xtelñilññenil1lfimifetleñlo2eñilq1fimxilet1lemdimfe5lrm1l1ñxe1lxelne2mdelf1515lxelv1ttilwlf1g1vt1n1lfim5dtr2x15l1lñ1lit2ññ1lxelrmlt2ilxel1sr15lx2141m15l7rel5el8tef282d1v1ml8itlrmlñefoilxel82ext15l8rñ2x15lvñ1mf15lwlemitqe5lfiqiloreni5l8teo25dit2fi5leñlqrmxilet1ld1mltef2emdel7relqrfo15lfi515lf1tef21mlxelmiqvtelwl81t1lqemf2im1tñ15lo1v21l7rel5eg1ñ1t215lfimleñlxexildixi5lñi5l1gi5l8itleñlqe5lxelq1t6ilrm1l41q2ñ21lxels2d1mi5lxe51tt181xi5l8ñ1md1v1l5rlf1t81lfetf1lxelñ1l1ñxe1lwlfimlrmlst1mxel1ñvitidilxel82di5lwld2qv1ñe5lx1v1ml1lfimifetlñi5lmreni5l2mnemdi5l8t2qetilññen1timleñl2q1mlrmls2d1milfit8rñemdilxelv1tv1lqimd1t16lwlq1mi5lxelsitt2iml7rel5el8te5emdilfimleñlmiqvtelxelqeñ7r21xe5lo26ilrm1ldtrfrñemd1lxeqi5dt1f2iml8rvñ2f1lxelñil7releñlq25qilññ1q1v1lñ1lifd1n1lq1t1n2ññ1lxelñi5l51v2i5l1ñ7r2q25d15lxelq1fexim21l4relxelf151lemlf151l1tt15dt1mxilxi5lñ2mside5lqed1ñ2fi5lwldix

## Busqueda de la clave

Empezaremos la busqueda de la clave adecuado, utilizando en el inicio una **clave** aleatoria que iremos cambiando por una  clave cercana, simplemente transponiendo dos caracteres seleccionados aleatoriamente.

In [52]:
clave0 = make_random_key()  # creamos una clave inicial
Clave0

{'a': 'l',
 'b': '2',
 'c': 'z',
 'd': 'c',
 'e': 's',
 'f': 'd',
 'g': 't',
 'h': 'w',
 'i': 'f',
 'j': 'j',
 'k': 'o',
 'l': '3',
 'm': 'a',
 'n': '5',
 'ñ': 'ñ',
 'o': 'n',
 'p': 'u',
 'q': '7',
 'r': ' ',
 's': 'g',
 't': 'y',
 'u': 'p',
 'v': 'q',
 'w': '4',
 'x': 'k',
 'y': 'i',
 'z': '0',
 '0': 'b',
 '1': '6',
 '2': 'x',
 '3': '8',
 '4': 'v',
 '5': 'r',
 '6': 'h',
 '7': '1',
 '8': 'e',
 '9': '9',
 ' ': 'm'}

In [53]:
decoded0 =code(cipher_text, clave0) # decodificamos
decoded0

'4thj7psxy7pso3pmt3psif3eb3sx9sm397b7eso3sitpk9x4k3eb7s39sh7f7e39sxtf39kxe7snt3eokxsjxnkxso3sf3h7foxfsxgt399xsbxfo3sf347bxs3esgt3sptsmxof3s97s99307sxsh7e7h3fs39sjk397s4xh7eo7s3fxs3eb7eh3pstexsx9o3xso3s03keb3shxpxpso3snxff7sashxyxnfx0xsh7epbftkoxpsxs9xs7fk99xso3stesfk7so3sxutxpsokxixexpsgt3sp3smf3hkmkbxnxesm7fstes93hj7so3smk3ofxpsmt9koxpsn9xehxpsas3e7f43psh747sjt307psmf3jkpb7fkh7ps39s4teo7s3fxsbxesf3hk3eb3sgt3s4thjxpsh7pxpshxf3hkxeso3se74nf3sasmxfxs43ehk7exf9xpsjxnkxsgt3sp3yx9xfkxpsh7es39so3o7sb7o7ps97psxy7psm7fs39s43pso3s4xfd7stexsix4k9kxso3sukbxe7pso3pxffxmxo7psm9xebxnxsptshxfmxsh3fhxso3s9xsx9o3xsash7estesufxeo3sx9n7f7b7so3smkb7psasbk4nx93psoxnxesxsh7e7h3fs97pset307pske03eb7psmfk43f7s9930xf7es39sk4xestesukbxe7sh7fmt93eb7so3snxfnxs47ebxfxdsas4xe7pso3su7ffk7esgt3sp3smf3p3eb7sh7es39se74nf3so3s439gtkxo3psjkd7stexsbftht93ebxso347pbfxhk7esmtn9khxso3s97sgt3s39s4kp47s99x4xnxs9xs7hbx0xs4xfx0k99xso3s97pspxnk7psx9gtk4kpbxpso3s4xh3o7ekxsit3so3shxpxs3eshxpxsxffxpbfxeo7so7ps9keu7b3ps43bx9kh7psasb7o

In [54]:
log_p0 = goodness(decoded0, bigram_table)  # evaluamos
log_p0

-31695.151555329383

In [55]:
pair = random.sample(alphabet, 2)
pair[0], pair[1]

('s', 'f')

In [56]:
clave0[pair[0]] ,clave0[pair[1]]

('u', 'h')

In [57]:
clave1 = clave0.copy()
clave1[pair[0]] = clave0[pair[1]]
clave1[pair[1]] = clave0[pair[0]]
clave1

{'a': '1',
 'b': 'q',
 'c': 'w',
 'd': 'b',
 'e': '3',
 'f': 'u',
 'g': 'y',
 'h': '5',
 'i': '7',
 'j': '2',
 'k': 'r',
 'l': 's',
 'm': 'e',
 'n': '0',
 'ñ': '9',
 'o': 'j',
 'p': 'l',
 'q': '4',
 'r': 't',
 's': 'h',
 't': 'f',
 'u': '8',
 'v': 'n',
 'w': 'a',
 'x': 'o',
 'y': ' ',
 'z': 'z',
 '0': 'ñ',
 '1': 'x',
 '2': 'k',
 '3': 'v',
 '4': 'i',
 '5': 'p',
 '6': 'd',
 '7': 'g',
 '8': 'm',
 '9': 'c',
 ' ': '6'}

In [58]:
decoded1 =code(cipher_text, clave1) # decodificamos
decoded1

'4tuj7psxy7pso3pmt3psif3eb3sx9sm397b7eso3sitpk9x4k3eb7s39su7f7e39sxtf39kxe7snt3eokxsjxnkxso3sf3u7foxfsxgt399xsbxfo3sf347bxs3esgt3sptsmxof3s97s99307sxsu7e7u3fs39sjk397s4xu7eo7s3fxs3eb7eu3pstexsx9o3xso3s03keb3suxpxpso3snxff7sasuxyxnfx0xsu7epbftkoxpsxs9xs7fk99xso3stesfk7so3sxhtxpsokxixexpsgt3sp3smf3ukmkbxnxesm7fstes93uj7so3smk3ofxpsmt9koxpsn9xeuxpsas3e7f43psu747sjt307psmf3jkpb7fku7ps39s4teo7s3fxsbxesf3uk3eb3sgt3s4tujxpsu7pxpsuxf3ukxeso3se74nf3sasmxfxs43euk7exf9xpsjxnkxsgt3sp3yx9xfkxpsu7es39so3o7sb7o7ps97psxy7psm7fs39s43pso3s4xfd7stexsix4k9kxso3shkbxe7pso3pxffxmxo7psm9xebxnxsptsuxfmxsu3fuxso3s9xsx9o3xsasu7esteshfxeo3sx9n7f7b7so3smkb7psasbk4nx93psoxnxesxsu7e7u3fs97pset307pske03eb7psmfk43f7s9930xf7es39sk4xesteshkbxe7su7fmt93eb7so3snxfnxs47ebxfxdsas4xe7pso3sh7ffk7esgt3sp3smf3p3eb7su7es39se74nf3so3s439gtkxo3psjkd7stexsbftut93ebxso347pbfxuk7esmtn9kuxso3s97sgt3s39s4kp47s99x4xnxs9xs7ubx0xs4xfx0k99xso3s97pspxnk7psx9gtk4kpbxpso3s4xu3o7ekxsit3so3suxpxs3esuxpxsxffxpbfxeo7so7ps9keh7b3ps43bx9ku7psasb7o

In [59]:
log_p1 = goodness(decoded1, bigram_table)  # evaluamos
log_p1

-31172.02538634292

En base a lo anterior ¿Aceptamos la nueva clave propuesta?

## Algoritmo completo

In [60]:
def transpose_random(key):
    pair = random.sample(alphabet, 2)
    new_key = dict(key)
    new_key[pair[0]] = key[pair[1]]
    new_key[pair[1]] = key[pair[0]]
    return new_key

def decode(cipher_text, bigram_table, iters = 10000, print_every = 1000):
    # 1. inicializamos con una clave aleatoria
    current_key = make_random_key()  

    for i in range(0, iters):  # en cada iteración utilizaremos una nueva clave

        # 2. decodificamos la secuencia con la clave
        decoded = code(cipher_text, current_key) 

        if i % print_every == 0:
            print(str(i) + "\t" + decoded + "\n")

        # 3. evaluamos la secuencia utilizando el modelo de bigramas 
        score = goodness(decoded, bigram_table)   
               
        # 4. proponemos una nueva clave "cercana"
        changed_key = transpose_random(current_key) 

        # 5. evaluamos la nueva clave decodificando la secuencia  
        changed_score = goodness(code(cipher_text, changed_key), bigram_table)  # evaluamos la secuencia con la nueva clave propuesta

        # 6.1 si la clave propuesta da mejores resultados nos quedamos con ella para la prox iteración
        if changed_score > score:   
            current_key = changed_key

        # damos oportunidad proporcional a la relación entre la probabilidad de las dos secuencias evaluadas 
        else:  
            diff = changed_score - score
            if math.log(random.random())  < diff:
                current_key = changed_key

    decoded = code(cipher_text, current_key)
    print("Final decoded: " + decoded)
    return decoded

In [66]:
decode(cipher_text, bigram_table, 10000)

0	uwm0a9l42a9lqv97wv9lc3vñgvl46l7v6agañlqvlcw9f64ufvñgalv6lma3añv6l4w3v6f4ñal1wvñqf4l041f4lqvl3vma3q43l4kwv664lg43qvl3vuag4lvñlkwvl9wl74q3vl6al66vdal4lmañamv3lv6l0fv6alu4mañqalv34lvñgañmv9lwñ4l46qv4lqvldvfñgvlm4949lqvl1433alolm424134d4lmañ9g3wfq49l4l64la3f664lqvlwñl3falqvl4pw49lqf4c4ñ49lkwvl9vl73vmf7fg414ñl7a3lwñl6vm0alqvl7fvq349l7w6fq49l164ñm49lolvña3uv9lmaual0wvda9l73v0f9ga3fma9lv6luwñqalv34lg4ñl3vmfvñgvlkwvluwm049lma949lm43vmf4ñlqvlñau13vlol7434luvñmfañ43649l041f4lkwvl9v24643f49lmañlv6lqvqalgaqa9l6a9l42a9l7a3lv6luv9lqvlu438alwñ4lc4uf6f4lqvlpfg4ña9lqv9433474qa9l764ñg414l9wlm4374lmv3m4lqvl64l46qv4lolmañlwñlp34ñqvl461a3agalqvl7fga9lolgfu146v9lq414ñl4lmañamv3l6a9lñwvda9lfñdvñga9l73fuv3al66vd43añlv6lfu4ñlwñlpfg4ñalma37w6vñgalqvl14314luañg4348lolu4ña9lqvlpa33fañlkwvl9vl73v9vñgalmañlv6lñau13vlqvluv6kwf4qv9l0f8alwñ4lg3wmw6vñg4lqvua9g34mfañl7w16fm4lqvl6alkwvlv6luf9ual664u414l64lamg4d4lu434df664lqvl6a9l941fa9l46kwfuf9g49lqvlu4mvqañf4lcwvlqvlm494lvñlm494l43349g34ñqalqa9l6fñpagv9luvg46fma9lolga

'muchos años despues frente al peloton de fusilamiento el coronel aureliano buendia habia de recordar avuella tarde remota en vue su padre lo llejo a conocer el hielo macondo era entonces una aldea de jeinte casas de barro y cañabraja construidas a la orilla de un rio de aguas diafanas vue se precipitaban por un lecho de piedras pulidas blancas y enormes como huejos prehistoricos el mundo era tan reciente vue muchas cosas carecian de nombre y para mencionarlas habia vue señalarias con el dedo todos los años por el mes de marzo una familia de gitanos desarrapados plantaba su carpa cerca de la aldea y con un grande alboroto de pitos y timbales daban a conocer los nuejos injentos primero llejaron el iman un gitano corpulento de barba montaraz y manos de gorrion vue se presento con el nombre de melvuiades hizo una truculenta demostracion publica de lo vue el mismo llamaba la octaja marajilla de los sabios alvuimistas de macedonia fue de casa en casa arrastrando dos lingotes metalicos y tod

# Otros ejemplos

In [76]:
# El principito. Tomado de https://archive.org/stream/ElPrincipitoAntoineDeSaintExupery/El_principito-_Antoine_De_Saint_Exupery_djvu.txt

plain_text = limpiar_texto(
    """Tengo poderosas razones para creer que el planeta del cual venía el principito era 
el asteroide B 612. Este asteroide ha sido visto sólo una vez con el telescopio en 
1909, por un astrónomo turco. 
Este astrónomo hizo una gran demostración de su descubrimiento en un congreso 
Internacional de Astronomía. Pero nadie le creyó a causa de su manera de vestir. 
Las personas mayores son así. Felizmente para la reputación del asteroide B 612, 
un dictador turco impuso a su pueblo, bajo pena de muerte, el vestido a la 
europea. Entonces el astrónomo volvió a dar cuenta de su descubrimiento en 1920 
y como lucía un traje muy elegante, todo el mundo aceptó su demostración. 
"""
)
cipher_text = code(plain_text, key)
cipher_text

'demsil8ixeti515lt16ime5l81t1lfteetl7releñl8ñ1med1lxeñlfr1ñlnem21leñl8t2mf282dilet1leñl15deti2xelvlcy le5del15deti2xelo1l52xiln25dil5iñilrm1lne6lfimleñldeñe5fi82ilemlykakl8itlrml15dtimiqildrtfile5del15dtimiqilo26ilrm1lst1mlxeqi5dt1f2imlxel5rlxe5frvt2q2emdilemlrmlfimste5il2mdetm1f2im1ñlxel15dtimiq21l8etilm1x2elñelftewil1lf1r51lxel5rlq1met1lxelne5d2tlñ15l8et5im15lq1wite5l5iml152l4eñ26qemdel81t1lñ1lte8rd1f2imlxeñl15deti2xelvlcy lrmlx2fd1xitldrtfil2q8r5il1l5rl8revñilv1pil8em1lxelqretdeleñlne5d2xil1lñ1lerti8e1lemdimfe5leñl15dtimiqilniñn2il1lx1tlfremd1lxel5rlxe5frvt2q2emdilemlyk alwlfiqilñrf21lrmldt1pelqrwleñes1mdeldixileñlqrmxil1fe8dil5rlxeqi5dt1f2im'

In [78]:
decode(cipher_text, bigram_table, 20000)

0	k7ñ361v6o7y6j4j1y4h6ñ7j1v4y41iy77y1e2717s1vs4ñ7k41o7s1i24s107ñp417s1vypñipvpk617y417s14jk7y6po71d1cq 17jk714jk7y6po71541jpo610pjk61j6s612ñ4107h1i6ñ17s1k7s7ji6vp617ñ1qbab1v6y12ñ14jky6ñ6r61k2yi617jk714jky6ñ6r615ph612ñ413y4ñ1o7r6jky4ip6ñ1o71j21o7ji2dyprp7ñk617ñ12ñ1i6ñ3y7j61pñk7yñ4ip6ñ4s1o714jky6ñ6rp41v7y61ñ4op71s71iy7x6141i42j41o71j21r4ñ7y41o7107jkpy1s4j1v7yj6ñ4j1r4x6y7j1j6ñ14jp1n7sphr7ñk71v4y41s41y7v2k4ip6ñ1o7s14jk7y6po71d1cq 12ñ1opik4o6y1k2yi61prv2j6141j21v27ds61d4g61v7ñ41o71r27yk717s107jkpo6141s4172y6v7417ñk6ñi7j17s14jky6ñ6r6106s0p6141o4y1i27ñk41o71j21o7ji2dyprp7ñk617ñ1qb a1x1i6r61s2ip412ñ1ky4g71r2x17s734ñk71k6o617s1r2ño614i7vk61j21o7r6jky4ip6ñ

1000	carze pemaielol ioderal poio biaai yta an pnoraco man bton varuo an piurbupuce aio an olcaieuma f w14 alca olcaieuma go lume vulce lene tro vad ber an canalbepue ar 10k0 pei tr olcierese ctibe alca olcierese gude tro zior maselciobuer ma lt malbtfiusuarce ar tr berziale urcairobueron ma olcieresuo paie romua na biaxe o botlo ma lt soraio

'tengo poderosas razones para creer jue el planeta del cual venia el principito era el asteroide b q10 este asteroide ha sido visto solo una vez con el telescopio en 1646 por un astronomo turco este astronomo hizo una gran demostracion de su descubrimiento en un congreso internacional de astronomia pero nadie le creyo a causa de su manera de vestir las personas mayores son asi felizmente para la reputacion del asteroide b q10 un dictador turco impuso a su pueblo baño pena de muerte el vestido a la europea entonces el astronomo volvio a dar cuenta de su descubrimiento en 1604 y como lucia un trañe muy elegante todo el mundo acepto su demostracion'

In [80]:
# La liebre y la tortuga. Tomado de https://psicologiaymente.com/cultura/fabulas-de-esopo

plain_text = limpiar_texto(
    """Un día una liebre orgullosa y veloz, vió como una tortuga caminaba por el camino y se le acercó. La liebre empezó a burlarse de la lentitud del otro animal y de la longitud de sus patas. Sin embargo, la tortuga le respondió que estaba segura de que a pesar de la gran velocidad de la liebre era capaz de ganarla en una carrera.
La liebre, segura de su victoria y considerando el reto imposible de perder, aceptó. Ambos pidieron a la zorra que señalara la meta, a lo que esta aceptó, al igual que al cuervo para que hiciera de juez.
Al llegar el día de la competición, al empezar la carrera la liebre y la tortuga salieron al mismo tiempo. La tortuga avanzaba sin detenerse, pero lentamente.
La liebre era muy veloz, y viendo que sacaba una gran ventaja a la tortuga decidió ir parándose y descansando de vez en cuando. Pero en una de las ocasiones la liebre se quedó dormida. La tortuga, poco a poco, siguió avanzando.
Cuando la liebre despertó, se encontró con que la tortuga estaba a punto de cruzar la meta. Aunque echó a correr fue demasiado tarde y finalmente la tortuga ganó la carrera".
Esta fábula nos enseña que el trabajo duro, la perseverancia, la constancia y el esfuerzo nos llevarán a nuestras metas, aunque sea poco a poco, si no nos rendimos. También nos permite ver cómo la arrogancia, la falta de constancia y el exceso de seguridad en uno mismo nos puede llevar a perder oportunidades y a no alcanzar nuestras metas.
"""
)
cipher_text = code(plain_text, key)
cipher_text

'rmlx21lrm1lñ2evtelitsrññi51lwlneñi6ln2ilfiqilrm1lditdrs1lf1q2m1v1l8itleñlf1q2milwl5elñel1fetfilñ1lñ2evteleq8e6il1lvrtñ1t5elxelñ1lñemd2drxlxeñlidtil1m2q1ñlwlxelñ1lñims2drxlxel5r5l81d15l52mleqv1tsilñ1lditdrs1lñelte58imx2il7rele5d1v1l5esrt1lxel7rel1l8e51tlxelñ1lst1mlneñif2x1xlxelñ1lñ2evtelet1lf1816lxels1m1tñ1lemlrm1lf1ttet1lñ1lñ2evtel5esrt1lxel5rln2fdit21lwlfim52xet1mxileñltedil2q8i52vñelxel8etxetl1fe8dil1qvi5l82x2etiml1lñ1l6itt1l7rel5eg1ñ1t1lñ1lqed1l1lñil7rele5d1l1fe8dil1ñl2sr1ñl7rel1ñlfretnil81t1l7relo2f2et1lxelpre6l1ñlññes1tleñlx21lxelñ1lfiq8ed2f2iml1ñleq8e61tlñ1lf1ttet1lñ1lñ2evtelwlñ1lditdrs1l51ñ2etiml1ñlq25qild2eq8ilñ1lditdrs1l1n1m61v1l52mlxedemet5el8etilñemd1qemdelñ1lñ2evtelet1lqrwlneñi6lwln2emxil7rel51f1v1lrm1lst1mlnemd1p1l1lñ1lditdrs1lxef2x2il2tl81t1mxi5elwlxe5f1m51mxilxelne6lemlfr1mxil8etilemlrm1lxelñ15lif152ime5lñ1lñ2evtel5el7rexilxitq2x1lñ1lditdrs1l8ifil1l8ifil52sr2il1n1m61mxilfr1mxilñ1lñ2evtelxe58etdil5elemfimdtilfiml7relñ1lditdrs1le5d1v1l1l8rmdilxelftr61tlñ1lqed1l1rm7relefoi

In [81]:
decode(cipher_text, bigram_table, 10000)

0	hknxp5nhk5nfpcaicngiwhffg 5nen0cfgjn0pgnogtgnhk5n1gi1hw5no5tpk5a5nbgincfno5tpkgnen cnfcn5ociognf5nfpcaicnctbcjgn5nahif5i cnxcnf5nfck1p1hxnxcfng1ign5kpt5fnenxcnf5nfgkwp1hxnxcn h nb515 n pkncta5iwgnf5n1gi1hw5nfcnic bgkxpgnuhcnc 15a5n cwhi5nxcnuhcn5nbc 5inxcnf5nwi5kn0cfgopx5xnxcnf5nfpcaicnci5no5b5jnxcnw5k5if5ncknhk5no5iici5nf5nfpcaicn cwhi5nxcn hn0po1gip5nenogk pxci5kxgncfnic1gnptbg pafcnxcnbcixcin5ocb1gn5tag nbpxpcigkn5nf5njgii5nuhcn cq5f5i5nf5ntc15n5nfgnuhcnc 15n5ocb1gn5fnpwh5fnuhcn5fnohci0gnb5i5nuhcnrpopci5nxcnvhcjn5fnffcw5incfnxp5nxcnf5nogtbc1popgkn5fnctbcj5inf5no5iici5nf5nfpcaicnenf5n1gi1hw5n 5fpcigkn5fntp tgn1pctbgnf5n1gi1hw5n505kj5a5n pknxc1ckci cnbcignfck15tck1cnf5nfpcaicnci5nthen0cfgjnen0pckxgnuhcn 5o5a5nhk5nwi5kn0ck15v5n5nf5n1gi1hw5nxcopxpgnpinb5i5kxg cnenxc o5k 5kxgnxcn0cjncknoh5kxgnbcigncknhk5nxcnf5 ngo5 pgkc nf5nfpcaicn cnuhcxgnxgitpx5nf5n1gi1hw5nbgogn5nbgogn pwhpgn505kj5kxgnoh5kxgnf5nfpcaicnxc bci1gn cnckogk1ignogknuhcnf5n1gi1hw5nc 15a5n5nbhk1gnxcnoihj5inf5ntc15n5hkuhcncor

'un dia una liebre orgullosa y veloz vio como una tortuga caminaba por el camino y se le acerco la liebre empezo a burlarse de la lentitud del otro animal y de la longitud de sus patas sin embargo la tortuga le respondio fue estaba segura de fue a pesar de la gran velocidad de la liebre era capaz de ganarla en una carrera la liebre segura de su victoria y considerando el reto imposible de perder acepto ambos pidieron a la zorra fue señalara la meta a lo fue esta acepto al igual fue al cuervo para fue xiciera de juez al llegar el dia de la competicion al empezar la carrera la liebre y la tortuga salieron al mismo tiempo la tortuga avanzaba sin detenerse pero lentamente la liebre era muy veloz y viendo fue sacaba una gran ventaja a la tortuga decidio ir parandose y descansando de vez en cuando pero en una de las ocasiones la liebre se fuedo dormida la tortuga poco a poco siguio avanzando cuando la liebre desperto se encontro con fue la tortuga estaba a punto de cruzar la meta aunfue ecxo